# Bikeshare数据集上的特征工程

1、	任务描述
请在Capital Bikeshare （美国Washington, D.C.的一个共享单车公司）提供的自行车数据上进行回归分析。训练数据为2011年的数据，要求预测2012年每天的单车共享数量。

原始数据集地址：http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset
1)	文件说明
day.csv: 按天计的单车共享次数
readme：数据说明文件

## 导入必要的工具包

In [1]:
# 数据读取及基本处理
import pandas as pd
import numpy as np

## 读入数据
训练集合测试集放在了一起，所以可以一起读进来

In [2]:
# 读入数据
train = pd.read_csv("day.csv")
#查看头5行的信息
train.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


### 字段说明

Instant记录号

Dteday：日期   

Season：季节（1=春天、2=夏天、3=秋天、4=冬天）

yr：年份，(0: 2011, 1:2012)

mnth：月份( 1 to 12)


holiday：是否是节假日

weekday：星期中的哪天，取值为0～6

workingday：是否工作日

1=工作日 （是否为工作日，1为工作日，0为非周末或节假日

weathersit：天气（1：晴天，多云 2：雾天，阴天 3：小雪，小雨 4：大雨，大雪，大雾）

temp：气温摄氏度

atemp：体感温度

hum：湿度

windspeed：风速

casual：非注册用户个数

registered：注册用户个数

cnt：给定日期（天）时间（每小时）总租车人数，响应变量y


### 查看数据信息

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
instant       731 non-null int64
dteday        731 non-null object
season        731 non-null int64
yr            731 non-null int64
mnth          731 non-null int64
holiday       731 non-null int64
weekday       731 non-null int64
workingday    731 non-null int64
weathersit    731 non-null int64
temp          731 non-null float64
atemp         731 non-null float64
hum           731 non-null float64
windspeed     731 non-null float64
casual        731 non-null int64
registered    731 non-null int64
cnt           731 non-null int64
dtypes: float64(4), int64(11), object(1)
memory usage: 91.4+ KB


## 数据探索

In [5]:
#对数据值型特征，用常用统计量观察其分布
train.describe()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,366.000000,2.496580,0.500684,6.519836,0.028728,2.997264,0.683995,1.395349,0.495385,0.474354,0.627894,0.190486,848.176471,3656.172367,4504.348837
std,211.165812,1.110807,0.500342,3.451913,0.167155,2.004787,0.465233,0.544894,0.183051,0.162961,0.142429,0.077498,686.622488,1560.256377,1937.211452
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,2.000000,20.000000,22.000000
25%,183.500000,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,0.337083,0.337842,0.520000,0.134950,315.500000,2497.000000,3152.000000
50%,366.000000,3.000000,1.000000,7.000000,0.000000,3.000000,1.000000,1.000000,0.498333,0.486733,0.626667,0.180975,713.000000,3662.000000,4548.000000
75%,548.500000,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,0.655417,0.608602,0.730209,0.233214,1096.000000,4776.500000,5956.000000
max,731.000000,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,0.861667,0.840896,0.972500,0.507463,3410.000000,6946.000000,8714.000000


In [7]:
#对类别型特征，观察其取值范围及直方图

#设置一个列表
categorical_features = ['season','mnth','weathersit','holiday']

#循环从列表中取值
for col in categorical_features:
    print '\n%s属性的不同取值和出现的次数'%col
    #从读取的数据中拿到对应的列值  和  总条数
    print train[col].value_counts()
    #把对应的列变为object类型
    train[col] = train[col].astype('object')


season属性的不同取值和出现的次数
3    188
2    184
1    181
4    178
Name: season, dtype: int64

mnth属性的不同取值和出现的次数
12    62
10    62
8     62
7     62
5     62
3     62
1     62
11    60
9     60
6     60
4     60
2     57
Name: mnth, dtype: int64

weathersit属性的不同取值和出现的次数
1    463
2    247
3     21
Name: weathersit, dtype: int64

holiday属性的不同取值和出现的次数
0    710
1     21
Name: holiday, dtype: int64


类别型特征的取值不多                                                                                 
除了holiday特征，其他特征的各个取值的样本数目也比较均衡                                                           
类别型特征可以采用独热编码（One hot encoding）/哑编码

## 特征工程

In [9]:
#categorical_features = ['season','mnth','weathersit','holiday']      上面的步骤已经设置好列表了

#从训练集中拿到这些训练数据
x_train_cat = train[categorical_features]
#对数据进行独热编码
x_train_cat = pd.get_dummies(x_train_cat)
#查看头5行信息
x_train_cat.head()

,season_1,season_2,season_3,season_4,mnth_1,mnth_2,mnth_3,mnth_4,mnth_5,mnth_6,...,mnth_8,mnth_9,mnth_10,mnth_11,mnth_12,weathersit_1,weathersit_2,weathersit_3,holiday_0,holiday_1
0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [10]:
#数值型变量预处理

#MinMaxScaler   讲特征转化到  0到1之间
from sklearn.preprocessing import MinMaxScaler

#0到1之间的算法
mn_x = MinMaxScaler()
#定义要从数据中拿出来的所有列名
numerical_features = ['temp','atemp','hum','windspeed']

#转换并且训练
temp = mn_x.fit_transform(train[numerical_features])
#train[numerical_features]  从读取的数据集中拿出指定列的所有值

#变为2维表显示
x_train_num = pd.DataFrame(data=temp, columns=numerical_features, index =train.index)
#data=temp  显示的数据
#columns=numerical_features  显示的字段名
#index =train.index  显示的坐标

#拿到前5行数据
x_train_num.head()

,temp,atemp,hum,windspeed
0,0.355170,0.373517,0.828620,0.284606
1,0.379232,0.360541,0.715771,0.466215
2,0.171000,0.144830,0.449638,0.465740
3,0.175530,0.174649,0.607131,0.284297
4,0.209120,0.197158,0.449313,0.339143


In [11]:
# 调用pd的拼接方法  把数据拼接起来(拼接必须是df)
x_train = pd.concat([x_train_cat, x_train_num, train['holiday'],  train['workingday']], axis = 1, ignore_index=False)
#显示头5行
x_train.head()

,season_1,season_2,season_3,season_4,mnth_1,mnth_2,mnth_3,mnth_4,mnth_5,mnth_6,...,weathersit_2,weathersit_3,holiday_0,holiday_1,temp,atemp,hum,windspeed,holiday,workingday
0,1,0,0,0,1,0,0,0,0,0,...,1,0,1,0,0.355170,0.373517,0.828620,0.284606,0,0
1,1,0,0,0,1,0,0,0,0,0,...,1,0,1,0,0.379232,0.360541,0.715771,0.466215,0,0
2,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0.171000,0.144830,0.449638,0.465740,0,1
3,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0.175530,0.174649,0.607131,0.284297,0,1
4,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0.209120,0.197158,0.449313,0.339143,0,1


In [12]:
# 调用pd的拼接方法  把数据拼接起来(拼接必须是df)
FE_train = pd.concat([train['instant'], x_train,  train['yr'],train['cnt']], axis = 1)
#变成csv文件   不要索引
FE_train.to_csv('FE_day.csv', index=False)
#显示头5行信息
FE_train.head()

,instant,season_1,season_2,season_3,season_4,mnth_1,mnth_2,mnth_3,mnth_4,mnth_5,...,holiday_0,holiday_1,temp,atemp,hum,windspeed,holiday,workingday,yr,cnt
0,1,1,0,0,0,1,0,0,0,0,...,1,0,0.355170,0.373517,0.828620,0.284606,0,0,0,985
1,2,1,0,0,0,1,0,0,0,0,...,1,0,0.379232,0.360541,0.715771,0.466215,0,0,0,801
2,3,1,0,0,0,1,0,0,0,0,...,1,0,0.171000,0.144830,0.449638,0.465740,0,1,0,1349
3,4,1,0,0,0,1,0,0,0,0,...,1,0,0.175530,0.174649,0.607131,0.284297,0,1,0,1562
4,5,1,0,0,0,1,0,0,0,0,...,1,0,0.209120,0.197158,0.449313,0.339143,0,1,0,1600


In [13]:
#查看信息
FE_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 30 columns):
instant         731 non-null int64
season_1        731 non-null uint8
season_2        731 non-null uint8
season_3        731 non-null uint8
season_4        731 non-null uint8
mnth_1          731 non-null uint8
mnth_2          731 non-null uint8
mnth_3          731 non-null uint8
mnth_4          731 non-null uint8
mnth_5          731 non-null uint8
mnth_6          731 non-null uint8
mnth_7          731 non-null uint8
mnth_8          731 non-null uint8
mnth_9          731 non-null uint8
mnth_10         731 non-null uint8
mnth_11         731 non-null uint8
mnth_12         731 non-null uint8
weathersit_1    731 non-null uint8
weathersit_2    731 non-null uint8
weathersit_3    731 non-null uint8
holiday_0       731 non-null uint8
holiday_1       731 non-null uint8
temp            731 non-null float64
atemp           731 non-null float64
hum             731 non-null float64
windspeed     